In [1]:
!unzip -q /content/img_cls_weather_dataset.zip

In [2]:
import os
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch

from PIL import Image
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch.nn as nn

In [3]:
def set_seed(seed):
    """
    Set the random seed for reproducibility across multiple libraries.

    Args:
        seed (int): The seed value to set.
    """
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


# Set the seed for reproducibility
seed = 59
set_seed(seed)

In [4]:
# Define the root directory for the dataset
root_dir = 'weather-dataset/dataset'

# Initialize lists to store image paths and labels
img_paths = []
labels = []

# Map class labels to their corresponding names
classes = {
    label_idx: class_name
    for label_idx, class_name in enumerate(
        sorted(os.listdir(root_dir))
    )
}

# Read all image paths and their corresponding labels from the dataset
for label_idx, class_name in classes.items():
    class_dir = os.path.join(root_dir, class_name)
    for img_filename in os.listdir(class_dir):
        img_path = os.path.join(class_dir, img_filename)
        img_paths.append(img_path)
        labels.append(label_idx)

In [5]:
# Define dataset splitting parameters
val_size = 0.2  # Proportion of the validation set
test_size = 0.125  # Proportion of the test set
is_shuffle = True  # Whether to shuffle the dataset during splitting

# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    img_paths, labels,
    test_size=val_size,
    random_state=seed,
    shuffle=is_shuffle
)

# Further split the training set into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X_train, y_train,
    test_size=test_size,
    random_state=seed,
    shuffle=is_shuffle
)

In [6]:
class WeatherDataset(Dataset):
    def __init__(self, X, y, transform=None):
        """
        Initialize the WeatherDataset.

        Args:
            X (list): List of image paths.
            y (list): List of corresponding labels.
            transform (callable, optional): Optional transform to apply to the images.
        """
        self.img_paths = X
        self.labels = y
        self.transform = transform

    def __len__(self):
        """Return the number of samples in the dataset."""
        return len(self.img_paths)

    def __getitem__(self, idx):
        """
        Retrieve an item from the dataset.

        Args:
            idx (int): Index of the sample to retrieve.

        Returns:
            Tuple: Transformed image, corresponding label.
        """
        img_path = self.img_paths[idx]
        img = Image.open(img_path).convert("RGB")

        if self.transform:
            img = self.transform(img)

        return img, self.labels[idx]

In [7]:
def transform(img, img_size=(224, 224)):
    """
    Transform the image for model input.

    Args:
        img (PIL.Image.Image): Image to transform.
        img_size (tuple): Desired size of the image (width, height).

    Returns:
        torch.Tensor: Transformed and normalized image.
    """
    img = img.resize(img_size)
    img = np.array(img)[..., :3]  # Ensure RGB format
    img = torch.tensor(img).permute(2, 0, 1).float()  # Convert to tensor and rearrange dimensions
    normalized_img = img / 255.0  # Normalize pixel values

    return normalized_img

In [8]:
# Create datasets
train_dataset = WeatherDataset(X_train, y_train, transform=transform)
val_dataset = WeatherDataset(X_val, y_val, transform=transform)
test_dataset = WeatherDataset(X_test, y_test, transform=transform)

# Define batch sizes
train_batch_size = 512
test_batch_size = 8

# Create data loaders
train_loader = DataLoader(
    train_dataset,
    batch_size=train_batch_size,
    shuffle=True
)

val_loader = DataLoader(
    val_dataset,
    batch_size=test_batch_size,
    shuffle=False
)

test_loader = DataLoader(
    test_dataset,
    batch_size=test_batch_size,
    shuffle=False
)

In [9]:
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        """
        Initialize a ResidualBlock.

        Args:
            in_channels (int): Number of input channels.
            out_channels (int): Number of output channels.
            stride (int, optional): Stride for the convolution. Defaults to 1.
        """
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_channels, out_channels, kernel_size=3, stride=stride, padding=1
        )
        self.batch_norm1 = nn.BatchNorm2d(out_channels)
        self.conv2 = nn.Conv2d(
            out_channels, out_channels, kernel_size=3, stride=1, padding=1
        )
        self.batch_norm2 = nn.BatchNorm2d(out_channels)
        self.downsample = nn.Sequential()

        if stride != 1 or in_channels != out_channels:
            self.downsample = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )
        self.relu = nn.ReLU()

    def forward(self, x):
        """
        Define the forward pass of the ResidualBlock.

        Args:
            x (torch.Tensor): Input tensor.

        Returns:
            torch.Tensor: Output tensor after passing through the block.
        """
        shortcut = x.clone()
        x = self.conv1(x)
        x = self.batch_norm1(x)
        x = self.relu(x)
        x = self.conv2(x)
        x = self.batch_norm2(x)
        x += self.downsample(shortcut)
        x = self.relu(x)
        return x

In [10]:
class ResNet(nn.Module):
    def __init__(self, residual_block, n_blocks_lst, n_classes):
        """
        Initialize the ResNet architecture.

        Args:
            residual_block (nn.Module): Residual block to use in the architecture.
            n_blocks_lst (list): List of the number of blocks in each layer.
            n_classes (int): Number of output classes.
        """
        super(ResNet, self).__init__()

        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.batch_norm1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU()
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        # Residual layers
        self.conv2 = self.create_layer(residual_block, 64, 64, n_blocks_lst[0], stride=1)
        self.conv3 = self.create_layer(residual_block, 64, 128, n_blocks_lst[1], stride=2)
        self.conv4 = self.create_layer(residual_block, 128, 256, n_blocks_lst[2], stride=2)
        self.conv5 = self.create_layer(residual_block, 256, 512, n_blocks_lst[3], stride=2)

        # Pooling and classification layers
        self.avgpool = nn.AdaptiveAvgPool2d(1)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(512, n_classes)

    def create_layer(self, residual_block, in_channels, out_channels, n_blocks, stride):
        """
        Create a layer consisting of multiple residual blocks.

        Args:
            residual_block (nn.Module): The residual block to use.
            in_channels (int): Number of input channels.
            out_channels (int): Number of output channels.
            n_blocks (int): Number of residual blocks in the layer.
            stride (int): Stride for the first block in the layer.

        Returns:
            nn.Sequential: A sequential container of residual blocks.
        """
        blocks = []
        # First block with downsampling if needed
        blocks.append(residual_block(in_channels, out_channels, stride))

        # Remaining blocks with the same input and output channels
        for _ in range(1, n_blocks):
            blocks.append(residual_block(out_channels, out_channels, stride=1))

        return nn.Sequential(*blocks)

    def forward(self, x):
        """
        Define the forward pass of the ResNet.

        Args:
            x (torch.Tensor): Input tensor.

        Returns:
            torch.Tensor: Output tensor.
        """
        x = self.conv1(x)
        x = self.batch_norm1(x)
        x = self.relu(x)
        x = self.maxpool(x)

        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)

        x = self.avgpool(x)
        x = self.flatten(x)
        x = self.fc1(x)

        return x

In [11]:
n_classes = len(list(classes.keys()))
device = 'cuda' if torch.cuda.is_available() else 'gpu'

# Initialize the model and move it to the appropriate device
model = ResNet(
    residual_block=ResidualBlock,
    n_blocks_lst=[2, 2, 2, 2],
    n_classes=n_classes
).to(device)

In [12]:
def evaluate(model, dataloader, criterion, device):
    """
    Evaluate the model's performance on a given dataset.

    Args:
        model (nn.Module): The model to evaluate.
        dataloader (DataLoader): DataLoader for the dataset.
        criterion (nn.Module): Loss function.
        device (str): Device to use ('cuda' or 'cpu').

    Returns:
        Tuple[float, float]: Loss and accuracy on the dataset.
    """
    model.eval()
    correct = 0
    total = 0
    losses = []

    with torch.no_grad():
        for inputs, labels in dataloader:
            inputs, labels = inputs.to(device), labels.to(device)

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            losses.append(loss.item())

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    loss = sum(losses) / len(losses)
    acc = correct / total

    return loss, acc

In [13]:
def fit(model, train_loader, val_loader, criterion, optimizer, device, epochs):
    """
    Train the model and validate on a given dataset.

    Args:
        model (nn.Module): The model to train.
        train_loader (DataLoader): DataLoader for the training set.
        val_loader (DataLoader): DataLoader for the validation set.
        criterion (nn.Module): Loss function.
        optimizer (torch.optim.Optimizer): Optimizer for training.
        device (str): Device to use ('cuda' or 'cpu').
        epochs (int): Number of training epochs.

    Returns:
        Tuple[List[float], List[float]]: Training losses and validation losses.
    """
    train_losses = []
    val_losses = []

    for epoch in range(epochs):
        batch_train_losses = []

        model.train()
        for idx, (inputs, labels) in enumerate(train_loader):
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            batch_train_losses.append(loss.item())

        train_loss = sum(batch_train_losses) / len(batch_train_losses)
        train_losses.append(train_loss)

        val_loss, val_acc = evaluate(model, val_loader, criterion, device)
        val_losses.append(val_loss)

        print(f"EPOCH {epoch + 1}:\tTrain loss: {train_loss:.4f}\tVal loss: {val_loss:.4f}\tVal Acc: {val_acc:.4f}")

    return train_losses, val_losses

In [14]:
# Define hyperparameters
lr = 1e-2  # Learning rate
epochs = 25  # Number of training epochs

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(
    model.parameters(),
    lr=lr
)

# Train the model and capture training and validation losses
train_losses, val_losses = fit(
    model=model,
    train_loader=train_loader,
    val_loader=val_loader,
    criterion=criterion,
    optimizer=optimizer,
    device=device,
    epochs=epochs
)

EPOCH 1:	Train loss: 1.8831	Val loss: 2.3817	Val Acc: 0.1151
EPOCH 2:	Train loss: 1.3722	Val loss: 2.6001	Val Acc: 0.1180
EPOCH 3:	Train loss: 1.2131	Val loss: 2.6408	Val Acc: 0.1966
EPOCH 4:	Train loss: 1.1069	Val loss: 1.8019	Val Acc: 0.4086
EPOCH 5:	Train loss: 1.0826	Val loss: 1.2681	Val Acc: 0.5688
EPOCH 6:	Train loss: 1.0109	Val loss: 1.1449	Val Acc: 0.6016
EPOCH 7:	Train loss: 0.9649	Val loss: 1.2789	Val Acc: 0.5608
EPOCH 8:	Train loss: 0.9162	Val loss: 1.2314	Val Acc: 0.5732
EPOCH 9:	Train loss: 0.9088	Val loss: 1.7625	Val Acc: 0.4261
EPOCH 10:	Train loss: 0.8480	Val loss: 0.9957	Val Acc: 0.6584
EPOCH 11:	Train loss: 0.8049	Val loss: 1.2934	Val Acc: 0.5747
EPOCH 12:	Train loss: 0.7786	Val loss: 1.4826	Val Acc: 0.5259
EPOCH 13:	Train loss: 0.7521	Val loss: 1.2397	Val Acc: 0.5819
EPOCH 14:	Train loss: 0.7545	Val loss: 2.1854	Val Acc: 0.3554
EPOCH 15:	Train loss: 0.7182	Val loss: 1.3373	Val Acc: 0.5674
EPOCH 16:	Train loss: 0.6823	Val loss: 1.9281	Val Acc: 0.5164
EPOCH 17:	Train l

In [15]:
# Evaluate the model on the validation dataset
val_loss, val_acc = evaluate(
    model=model,
    dataloader=val_loader,
    criterion=criterion,
    device=device
)

# Evaluate the model on the test dataset
test_loss, test_acc = evaluate(
    model=model,
    dataloader=test_loader,
    criterion=criterion,
    device=device
)

# Print evaluation results
print("Evaluation on validation and test datasets:")
print(f"Validation accuracy: {val_acc:.4f}, Validation loss: {val_loss:.4f}")
print(f"Test accuracy: {test_acc:.4f}, Test loss: {test_loss:.4f}")

Evaluation on validation and test datasets:
Validation accuracy: 0.3795, Validation loss: 3.0551
Test accuracy: 0.3464, Test loss: 3.1849
